# 🤖 Janus + Azure OpenAI Assistant (Minimal Version)
This notebook provides a simple query-response interface using `input()` and `print()`.

👉 Type `exit` to quit.

In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain_community.chat_models import AzureChatOpenAI
from langchain.chat_models import init_chat_model
from ai_tools import JanusQueryTool

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"

with open("ai_context.txt", "r") as f:
    ai_context = f.read()

print("Creating LLM...")
llm = init_chat_model(
    model="gpt-4o",
    model_provider="azure_openai"
)
print("LLM created")

janus_tool = JanusQueryTool()
tools = [
    Tool(
        name="JanusQueryTool",
        func=janus_tool.run,
        description="Run SQL-like queries on Janus data mesh using Trino SQL."
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    handle_parsing_errors=True
)

In [ ]:
print("\n🧠 Janus Assistant Ready! Type your questions below. Type 'exit' to quit.\n")

while True:
    query = input("❓ Query: ")
    if query.lower() == "exit":
        print("👋 Session ended. Goodbye!")
        break
    response = agent.run(
        f"Use this context to help you answer the query.\n\nContext:\n{ai_context}\n\nQuery:\n{query}"
    )
    print("💬 Response:", response, "\n")